In [2]:
import requests

In [3]:
url="https://www.ptt.cc/bbs/StupidClown/index.html"

In [4]:
res=requests.get(url)

In [5]:
res.encoding

'utf-8'

In [6]:
print res.text

<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8" />
		

<meta name="viewport" content="width=device-width">

<title>看板 StupidClown 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/v2.13/bbs-common.css" />
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/v2.13/bbs.css" media="screen" />
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/v2.13/pushstream.css" media="screen" />
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/v2.13/bbs-print.css" media="print" />


<script src="//ajax.googleapis.com/ajax/libs/jquery/2.1.1/jquery.min.js"></script>
<script src="//images.ptt.cc/v2.13/bbs.js"></script>


		

<script type="text/javascript">

  var _gaq = _gaq || [];
  _gaq.push(['_setAccount', 'UA-32365737-1']);
  _gaq.push(['_setDomainName', 'ptt.cc']);
  _gaq.push(['_trackPageview']);

  (function() {
    var ga = document.createElement('script'); ga.type = 'text/javascript'; ga.async = true;
    ga.src = ('https:' ==

In [7]:
from pyquery import PyQuery 

In [8]:
s=PyQuery(res.text)

In [9]:
S=s('div.pull-right a.btn:nth-child(2)')#下一頁

In [10]:
S.attr('href')

'/bbs/StupidClown/index2506.html'

In [11]:
import re

###把數字取出來

In [12]:
p=re.compile("\d{4}")
m=p.search('/bbs/StupidClown/index2496.html')
page=m.group()#group回傳連接到的部分
#page
m

In [13]:
m=S.attr('href').replace(page,str(int(page)+1))
m

'/bbs/StupidClown/index2506.html'

In [14]:
ptt="https://www.ptt.cc"#網址

In [15]:
M=ptt+m

In [16]:
M

'https://www.ptt.cc/bbs/StupidClown/index2506.html'

測試單頁，取出每一篇

In [17]:
res1=requests.get("https://www.ptt.cc/bbs/StupidClown/index2497.html")
S1=PyQuery(res1.text)
S1('div.r-ent')
#S1('div.nrec span')

[<div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>, <div.r-ent>]

顯示推文數，並忽略刪除文章和內含[問卷]跟[集氣]
把推爆的文章存起來##rec

In [18]:
def rec(R):#R=S1('div.r-ent')篇的集合
    c=[]
    for i in R:
        S2=PyQuery(i)
        S2('div.nrec span')
        if  len(S2('div.nrec span'))!=0:
            #print S2('div.nrec span')[0].text
            if S2('div.nrec span')[0].text == u'爆':
                #print S2('div.title a').text()
                if filter(lambda x: x in S2('div.title a').text(),[u"[問卷]",u"[集氣]"])==[]:
                   c.append(S2)
    return c

In [19]:
c=[]
c.extend(rec(S1('div.r-ent')))
print c

[[<div.r-ent>]]


找出三篇"爆"的文章

In [20]:
def three(R):
    """
    Where site R putting url. 
    """
    c=[]
    res=requests.get(R)#讀入首頁
    s=PyQuery(res.text)#用pyquery帶入
    S=s('div.pull-right a.btn:nth-child(2)')#查到擺放“上一頁”的選項
    T=S.attr('href')#上一頁的連結
    p=re.compile("\d{4}")#re讀入四位數字型態
    t=p.search(S.attr('href'))#尋找連結中符合型態
    page=t.group()#把符合型態的取出
    T=T.replace(page,str(int(page)+2))
    page=str(int(page)+2)#首頁的頁數加一
    while len(c)<3:
        T=T.replace(page,str(int(page)-1))#置換成前一頁
        M=ptt+T#網址
    
        Res2=requests.get(M)#請求本頁
        S3=PyQuery(Res2.text)#用PyQuery帶入
        
        K=rec(S3('div.r-ent'))
        c.extend(K)#新增本頁的"爆"文
        page=str(int(page)-1)#把page改成上一頁的頁碼
        c=c[:3]#不超過三篇
    for i in c:
        print i('div.title a').text()+" "+ptt+i('div.title a').attr('href')
    return c

##抓到三篇

In [21]:
c=three(url)

[無言] 聽說~最近的紅茶有問題~~~ https://www.ptt.cc/bbs/StupidClown/M.1430258699.A.C3C.html
[無言] 最長的一夜，演技派蟑螂 https://www.ptt.cc/bbs/StupidClown/M.1430151883.A.3DF.html
[大哭] 鑰匙掉了 https://www.ptt.cc/bbs/StupidClown/M.1430132425.A.3C8.html


<h1 style="text-align:center;color:#228822">再來讓他每天都跑</h1>

In [22]:
#!/usr/bin/env python 用python跑

In [162]:
%%writefile ptt_stupid.py
#!/usr/bin/env python 
# -*- coding: UTF-8 -*-
import requests
import os
url="https://www.ptt.cc/bbs/StupidClown/index.html"
ptt="https://www.ptt.cc"
from pyquery import PyQuery
import re
def rec(R):#R=S1('div.r-ent')
    c=[]
    for i in R:
        S2=PyQuery(i)
        if  len(S2('div.nrec span'))!=0 and S2('div.title a').text()!="":
            if S2('div.nrec span')[0].text == u'爆':
                    if filter(lambda x: x in S2('div.title a').text(),[u"[問卷]",u"[集氣]"])==[]:
                        c.append(S2)
    return c
def three(R):
    """
    Where site R putting url. 
    """
    c=[]
    res=requests.get(R)
    s=PyQuery(res.text)
    S=s('div.pull-right a.btn:nth-child(2)')
    T=S.attr('href')
    p=re.compile("\d{4}")
    t=p.search(S.attr('href'))
    page=t.group()
    T=T.replace(page,str(int(page)+2))
    page=str(int(page)+2)
    while len(c)<5:
        T=T.replace(page,str(int(page)-1))
        M=ptt+T
        Res2=requests.get(M)
        S3=PyQuery(Res2.text)
        
        K=rec(S3('div.r-ent'))
        c.extend(K)
        page=str(int(page)-1)
        #c=c[:5]
    SS1="<!DOCTYPE html><html><header><meta charset=\"utf-8\"/></header><body>"
    SS2=[]
    for i in c:
        SS2.append("<a style=\"font-size:40px\" href=\""+ptt+i('div.title a').attr('href')+"\">"+i('div.title a').text()+"</a>"+"</br>")
    SS3="</h1></body></html>"
    SS=SS1
    for i in range(len(c)):
        SS+=SS2[i]
    SS=SS+SS3
    f=open("foo.html","w")
    print >>f,SS.encode('utf-8')
    f.close()
    return c
three(url)

os.system("open foo.html")

Overwriting ptt_stupid.py


In [157]:
from pyquery import PyQuery

In [152]:
rm ptt_stupid.py

In [153]:
ls

20150417LiveDemoNotes.ipynb  20150427amazon.ipynb         hi
20150422yahoonews.ipynb      Homework1.ipynb              yahoonews.html
20150424youtube.ipynb        Homework1_1.ipynb            yahoonews.txt
20150426ptt.ipynb            foo.html                     youtube/


In [154]:
pwd

u'/Users/annie/AG/PyCon2015-Crawlers/MingYiNotes'

In [1]:
%run ptt_stupid.py #可以直接show在網頁上了

#每天都讓他跑的執行指令crontab
[教學](http://linux.vbird.org/linux_basic/0430cron.php)

In [215]:
#crontab [-u username] [-l|-e|-r]
#代表意義	分鐘	小時	日期	月份	週	指令
#數字範圍	0-59	0-23	1-31	1-12	0-7	呀就指令啊

#cd ~/bin/  
#在bin底下加入執行檔

#sudo su - (switch user) 有需要再用
#exit (不然在root太恐怖了)

#ls -l
#chmod 444 檔案名 [改變使用者權限 (ugo user group other 或是a all, rwx read write exe, ug+w 或者二進位去算 777 444)]

#!crontab -e
#00 * * * * /Users/annie/anaconda/bin/python /Users/annie/bin/ptt_stupid.py

```
SS1="<!DOCTYPE html><html><header><meta charset=\"utf-8\"/></header><body>"
SS2=[]
for i in c:
            SS2.append("<a style=\"font-size:40px\" href=\""+ptt+i('div.title a').attr('href')+"\">"+i('div.title a').text()+"</a>")
SS3="</h1></body></html>"
SS=SS1+SS2[0]+"<br>"+SS2[1]+"<br>"+SS2[2]+SS3
f=open("foo.html","w")
print >>f,SS.encode('utf-8')
f.close()
```

In [245]:
!cat foo.html

<!DOCTYPE html><html><header><meta charset="utf-8"/></header><body><a style="font-size:40px" href="https://www.ptt.cc/bbs/StupidClown/M.1430062647.A.226.html">[大哭] 應屆新鮮人第一次找工作</a><br><a style="font-size:40px" href="https://www.ptt.cc/bbs/StupidClown/M.1430040862.A.81D.html">[聽錯] 總統是第幾級產業?</a><br><a style="font-size:40px" href="https://www.ptt.cc/bbs/StupidClown/M.1429871971.A.F3A.html">[眼殘] 玩命光頭7</a></h1></body></html>


In [196]:
f=open("foo.html")

u'/Users/annie/AG/PyCon2015-Crawlers/MingYiNotes'